Install Pre Requisites

In [1]:
%pip install pybamm
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Genarting Battery SOC prediction dataset with prelabeled SOC using Coulomb Counting for 40 Variant (Per 1% Capacity/AH Degredation)

In [7]:
import pybamm
import pandas as pd
from pybamm import BaseModel
from IPython.display import display

## [1] Doyle Fuller Newman [2] Single Particle Model (SPM) [3] Single Particle Model with Electrolyte (SPMe)
availableModel = ["DFN", "SPM", "SPME"]
selectedModel = availableModel[2]
model: BaseModel

# Choose model
match selectedModel:
    case "DFN":
        model = pybamm.lithium_ion.DFN()
    case "SPM":
        model = pybamm.lithium_ion.SPM()
    case "SPME":
        model = pybamm.lithium_ion.SPMe(options={"thermal": "lumped", "SEI": "constant"})

# List of capacities from 5.0 Ah to 3.05 Ah in 1% steps
capacity_variants = [round(5.0 * (1 - i / 100), 2) for i in range(1)]

# Final DataFrame to store all results
all_results = pd.DataFrame()

# Loop through all capacity variants
for variant_num, cap in enumerate(capacity_variants, start=1):
    print(f"Running simulation for Variant {variant_num} with Q = {cap} Ah")

    selectedInputParameter = pybamm.ParameterValues("Chen2020")
    selectedInputParameter.update({
        "Nominal cell capacity [A.h]": cap
    })
    
    experiment = pybamm.Experiment(
        [
            (
                "Charge at 2C until 4.2 V",
                "Discharge at C/6 until 2.5 V"
            )
        ],
        period="5 second"
    )

    sim = pybamm.Simulation(model, experiment=experiment, parameter_values=selectedInputParameter)
    
    try:
        sim.solve()
    except Exception as e:
        print(f"Simulation failed for Q = {cap} Ah: {e}")
        continue  # skip to the next variant

    # Prepare data collection
    output_variables = ["Time [s]", "Current [A]", "Voltage [V]", "X-averaged cell temperature [K]",
                        "BatteryCapacity", "TimeDiff", "CapacityDiff", "SOC", "X-averaged negative particle stoichiometry"]
    
    data = {}
    num_rows = len(sim.solution["Time [min]"].entries)

    for var in output_variables:
        if var == "TimeDiff":
            timeDiffValues = [0] + [
                sim.solution["Time [s]"].entries[i] - sim.solution["Time [s]"].entries[i - 1]
                for i in range(1, num_rows)
            ]
            data[var] = timeDiffValues
        elif var == "BatteryCapacity":
            data[var] = [cap] * num_rows
        elif var == "CapacityDiff":
            capacityDiffValues = [0]
            for i in range(1, num_rows):
                deltaCapacity = sim.solution["Current [A]"].entries[i] * (timeDiffValues[i] / 3600) * -1
                capacityDiffValues.append(deltaCapacity)
            data[var] = capacityDiffValues
        elif var == "SOC":
            soc_values = [1]
            for i in range(1, num_rows):
                updated_soc = soc_values[i - 1] + (data["CapacityDiff"][i] / cap)
                soc_values.append(max(0, min(1, updated_soc)))
            data[var] = soc_values
        else:
            data[var] = sim.solution[var].entries

    df = pd.DataFrame(data)
    df["Variant"] = variant_num
    all_results = pd.concat([all_results, df], ignore_index=True)

# Save to CSV
all_results.to_csv("batch_battery_simulation.csv", index=False)
display(all_results)


Running simulation for Variant 1 with Q = 5.0 Ah


ValueError: Per-column arrays must each be 1-dimensional